# Globus Groups API

Let's see how to automate Globus groups using the Groups API.
1. We'll first login as ourselves, so everything we do is owned by us.
2. We'll check our own current group memberships.
3. We'll create a group.
4. We'll list membership of the group.
5. We'll add someone to the group and again list membership.
7. We'll remove someone from the group.
8. We'll delete the group.

In [ ]:
import json  # just so we can pretty-print response data
import globus_sdk

# I first need to log in as a human so I can do things as myself. 
# Because this is a notebook, the LOGIN_CLIENT_ID doesn't get to have a secret.
LOGIN_CLIENT_ID = "..."

# Create a client object for user login flows (to allow us do things as the human user)
myloginclient = globus_sdk.NativeAppAuthClient(LOGIN_CLIENT_ID)

In [ ]:
# Now I need to gather up all the API scopes my application needs me to consent to. 

# Auth scopes for obtaining human identity info
openid_scope = globus_sdk.scopes.data.AuthScopes.openid
email_scope = globus_sdk.scopes.data.AuthScopes.email
profile_scope = globus_sdk.scopes.data.AuthScopes.profile

# Groups API scope
group_scope = globus_sdk.scopes.data.GroupsScopes.all

requested_scopes = [openid_scope, email_scope, profile_scope, group_scope]

## Login time! 

Let's login the human using this notebook and get permission to manage groups for him/her.

In [ ]:
# Now that we know what we're asking permission to do, log the human in!

myloginclient.oauth2_start_flow(requested_scopes=requested_scopes, refresh_tokens=False)
print(f"Login Here:\n\n{myloginclient.oauth2_get_authorize_url()}")
print("\nIMPORTANT NOTE: the link above can only be used once!")
print("If login or a later step in the flow fails, you must execute this cell again to generate a new link.")
auth_code = input("PASTE YOUR CODE HERE> ")
tokens = myloginclient.oauth2_exchange_code_for_tokens(auth_code).by_resource_server
print("Tokens Received!")

In [ ]:
# Let's take a look at the tokens we received for this human
human_auth_data = tokens["auth.globus.org"]
human_groups_data = tokens["groups.api.globus.org"]

print("Here's how the Auth API token is delivered to us:")
print(json.dumps(human_auth_data, indent=2))
print("\nHere's how the Groups API token is delivered to us:")
print(json.dumps(human_groups_data, indent=2))

In [ ]:
# Now let's use the Groups token to create a Groups Client that lets me use the Groups API.

groups_tokens = tokens["groups.api.globus.org"]

# construct an AccessTokenAuthorizer and use it to construct the
# TransferClient
groups_client = globus_sdk.GroupsClient(
    authorizer=globus_sdk.AccessTokenAuthorizer(groups_tokens["access_token"])
)

# And let's get an AuthClient for use later in the exercises...
human_auth_access_token = human_auth_data['access_token']
human_auth_authorizer = globus_sdk.AccessTokenAuthorizer(human_auth_access_token)
human_auth_client = globus_sdk.AuthClient(authorizer=human_auth_authorizer)

## List all groups I belong to

The following code uses the Groups Client we created above to `get_my_groups()` and
print a CSV with group UUID, Display Name, Group Type, Session Enforcement Type, and my Role in the group.

In [ ]:
# print out in CSV format
# note that 'name' could have a comma in it, so this is slightly unsafe
print("ID,Name,Type,Session Enforcement,Roles")
for group in groups_client.get_my_groups():
    # parse the group to get data for output
    if group.get("enforce_session"):
        session_enforcement = "strict"
    else:
        session_enforcement = "not strict"
    roles = ",".join({m["role"] for m in group["my_memberships"]})

    print(
        ",".join(
            [
                group["id"],
                group["name"],
                group["group_type"],
                session_enforcement,
                roles,
            ]
        )
    )

## Create a new group

In [ ]:
# First, we build a group definition.

# Replace YOURNAME with your name below!
group_def = {"name": "YOURNAME's Very First API-created Group"}

In [ ]:
# Now, create the group.
result = groups_client.create_group(group_def)

# And print the group that got created!
print(json.dumps(result.data, indent=2))

In [ ]:
# We're going to grab the group's UUID and use it for subsequent exercises.

my_group_uuid = result.data["id"]

In [ ]:
# The group's members are listed in the response above. But if we want to get them
# later, here's how to do it. 

result = groups_client.get_group(my_group_uuid,include="memberships")
print(json.dumps(result.data, indent=2))

## Add a new member to your group

For managing a group's membership, we use the Globus SDK's `GroupManager` high-level client wrapper.

You can invite people to groups, and that's preferred because it gives them the
opportunity to decline membership. But if you really want to add without asking, or
if it's an application identity you're adding, you can do it this way. 

**Note: You're adding a Globus ID that's rarely used and isn't linked to any other IDs.**

In [ ]:
from globus_sdk import GroupsManager

mymanager = GroupsManager(groups_client)

# Get the UUID of the identity I want to add to the group
result = human_auth_client.get_identities(usernames="awesome@globusid.org")
for identity in result:
    awesome_id = identity["id"]
print("We're going to add {}.".format(awesome_id))

In [ ]:
# Now let's actually add it to the group.

result = mymanager.add_member(my_group_uuid,awesome_id,role='member')
print("Adding awesome@globusid.org to the group...")
print(json.dumps(result.data, indent=2))
print("\nHere's the new group membership:")
result = groups_client.get_group(my_group_uuid,include="memberships")
print(json.dumps(result.data, indent=2))

## Remove a member from the group

Again, we'll use the GroupManager wrapper.

**Notice that the group still lists the former member with the `"removed"` status!**

When checking group memberships, you must be careful to check the `status` field.

In [ ]:
result = mymanager.remove_member(my_group_uuid,awesome_id)
print("Removing awesome@globusid.org from the group...")
print(json.dumps(result.data, indent=2))
print("\nHere's the new group membership:")
result = groups_client.get_group(my_group_uuid,include="memberships")
print(json.dumps(result.data, indent=2))

## Delete the group

Ok, we've had our fun. Now let's delete this group so it doesn't create any
surprises for us later.

In [ ]:
result = groups_client.delete_group(my_group_uuid)
print("Deleting the group...")
print(json.dumps(result.data, indent=2))